In [1]:
import os
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
mlflow.set_tracking_uri("http://mlflow-service:5000")
mlflow.set_experiment("iris_evaluate_example")

<Experiment: artifact_location='s3://mlops-2025/mlflow/628412400198569280', creation_time=1759310236486, experiment_id='628412400198569280', last_update_time=1759310236486, lifecycle_stage='active', name='iris_evaluate_example', tags={}>

In [3]:
iris = load_iris(as_frame=True)
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [5]:
X.head(2)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2


In [6]:
y.head(2)

0    0
1    0
Name: target, dtype: int64

In [11]:
evaluate_df = pd.concat([X_test, y_test], axis=1)

In [13]:
evaluate_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
73,6.1,2.8,4.7,1.2,1
18,5.7,3.8,1.7,0.3,0
118,7.7,2.6,6.9,2.3,2
78,6.0,2.9,4.5,1.5,1
76,6.8,2.8,4.8,1.4,1


In [12]:
with mlflow.start_run() as run:
    clf = RandomForestClassifier(n_estimators=50, random_state=42)
    clf.fit(X_train, y_train)

    # логируем модель
    model_info = mlflow.sklearn.log_model(
        sk_model=clf,
        artifact_path="model",
        registered_model_name="iris_rf_model"
    )

    # evaluate → метрики и артефакты
    result = mlflow.evaluate(
        model=model_info.model_uri,
        data=evaluate_df,
        targets="target",
        model_type="classifier",
        evaluators=["default"]
    )

    # руками продублируем ключевые метрики, чтобы они были "видны" в Registry
    #mlflow.log_metrics(result.metrics)

    run_id = run.info.run_id

/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'iris_rf_model' already exists. Creating a new version of this model...
2025/10/01 15:19:53 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris_rf_model, version 30
Created version '30' of model 'iris_rf_model'.
2025/10/01 15:19:53 WARNING m

/opt/conda/lib/python3.11/site-packages/mlflow/types/utils.py:406: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/10/01 15:19:55 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2025/10/01 15:19:55 INFO mlflow.models.evaluation.default_evaluator: The evaluation dataset is inferred as multiclass dataset, number of classes is in

<Figure size 1050x700 with 0 Axes>

In [14]:
from mlflow import MlflowClient

In [15]:
client = MlflowClient()
latest = client.get_latest_versions("iris_rf_model", stages=["None"])[0]

/tmp/ipykernel_154/214642199.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions("iris_rf_model", stages=["None"])[0]


In [16]:
client.transition_model_version_stage(
    name="iris_rf_model",
    version=latest.version,
    stage="Staging"
)

/tmp/ipykernel_154/3618906252.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1759331993759, current_stage='Staging', description='', last_updated_timestamp=1759332280176, name='iris_rf_model', run_id='987c401a57794191ab5e75b0ebfd07b7', run_link='', source='s3://mlops-2025/mlflow/628412400198569280/987c401a57794191ab5e75b0ebfd07b7/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='30'>

mlflow models serve --model-uri models:/iris_rf_model/Staging --host 0.0.0.0 --port 5001 --no-conda


curl -X POST http://127.0.0.1:5001/invocations \
  -H 'Content-Type: application/json' \
  -d '{"inputs": [[5.1, 3.5, 1.4, 0.2]]}'
